In [ ]:
# Tarvittavien kirjastojen tuonti
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
# Datan lataaminen ja esikäsittely
def lataa_ja_muunna_audio(audio_tiedosto, näytteenottotaajuus=22050):
    aaltomuoto, sr = librosa.load(audio_tiedosto, sr=näytteenottotaajuus)
    return aaltomuoto, sr

In [ ]:
def luo_spektrogrammi(aaltomuoto, sr, hop_length=512):
    S = librosa.feature.melspectrogram(aaltomuoto, sr=sr, n_fft=2048, hop_length=hop_length)
    S_DB = librosa.power_to_db(S, ref=np.max)
    return S_DB

In [ ]:
# Oletetaan, että sinulla on lista kappalepolkuja ja vastaavia tanssilajilabeleita
kappalepolut = ['polku1.mp3', 'polku2.mp3', ...]
tanssilajilabelit = ['salsa', 'tango', ...]

# Ladataan ja esikäsitellään data
spektrogrammit = []
for polku in kappalepolut:
    aaltomuoto, sr = lataa_ja_muunna_audio(polku)
    spektrogrammi = luo_spektrogrammi(aaltomuoto, sr)
    spektrogrammit.append(spektrogrammi)

In [ ]:
# Muunnetaan lista numpy-taulukoksi
spektrogrammit_np = np.array(spektrogrammit)

# Labelien käsittely
label_encoder = LabelEncoder()
tanssilaji_numerot = label_encoder.fit_transform(tanssilajilabelit)
tanssilaji_one_hot = to_categorical(tanssilaji_numerot)

# Jaa data opetus- ja testisetteihin
X_train, X_test, y_train, y_test = train_test_split(spektrogrammit_np, tanssilaji_one_hot, test_size=0.2, random_state=42)


In [ ]:
# CNN-mallin määrittely
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(y_train.shape[1], activation='softmax')
])

In [ ]:
# Mallin kokoaminen
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Mallin kouluttaminen
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
# Mallin suorituskyvyn evaluointi
plt.plot(history.history['accuracy'], label='Opetustarkkuus')
plt.plot(history.history['val_accuracy'], label='Validointitarkkuus')
plt.xlabel('Epoch')
plt.ylabel('Tarkkuus')
plt.legend()
plt.show()